In [1]:
import requests
import pprint
from bs4 import BeautifulSoup
from datetime import datetime as dt
import pandas as pd
import csv
import os

In [2]:
#the analyzer - a basic example of how to use this data

In [3]:
os.getcwd()
os.chdir('C:/Users/Mac/Desktop/Investing/Financial Data')

In [7]:
#Reading and wrangling file

ticker_list_tsxv = []
stock_data_df = pd.read_csv('Venture_Data_Final - Copy.csv')
stock_data_df

stock_data_df = stock_data_df.drop('Unnamed: 0', 1)

In [8]:
#identify rows with no data on shares outstanding 

indexNames = stock_data_df[stock_data_df['Shares Outstanding'] < .001].index
stock_data_df.drop(indexNames , inplace=True)

Int64Index([  87,   88,  102,  156,  184,  202,  268,  356,  363,  377,  399,
             404,  418,  430,  442,  477,  480,  496,  498,  530,  584,  598,
             608,  651,  653,  656,  670,  679,  700,  701,  751,  752,  755,
             781,  782,  794,  814,  815,  838,  847,  857,  862,  869,  875,
             878,  880,  888,  946,  952,  997, 1014, 1045, 1048, 1105, 1142,
            1149, 1154, 1157, 1218, 1267, 1320, 1356, 1377, 1378, 1379, 1412,
            1452, 1464, 1468, 1490, 1491, 1496, 1529, 1543, 1544, 1549, 1551,
            1563, 1569, 1591],
           dtype='int64')


In [6]:
#transform data to account for thousands

stock_data_df['Total Cash'] = stock_data_df['Total Cash'].apply(lambda x: x*1000)
stock_data_df['Total Liabilities'] = stock_data_df['Total Liabilities'].apply(lambda x: x*1000)
stock_data_df['Total Revenue'] = stock_data_df['Total Revenue'].apply(lambda x: x*1000)
stock_data_df['Operating Cash Flow'] = stock_data_df['Operating Cash Flow'].apply(lambda x: x*1000)

In [8]:
#calculate net-net per share value of each company;
#subtract and divide by price to get relative indicator of a company's attractiveness relative to net cash position and price
#finally, sort by those companies with the highest net-net position relative to price

pd.set_option('display.float_format', lambda x: '%.3f' % x)
stock_data_df['Net Net per Share'] = (stock_data_df['Total Cash'] - stock_data_df['Total Liabilities'])/stock_data_df['Shares Outstanding']
stock_data_df['NNPS less Price as % of Price'] = (stock_data_df['Net Net per Share'] - stock_data_df['Price'])/stock_data_df['Price']

stock_data_df = stock_data_df.sort_values(by = ['NNPS less Price as % of Price'], ascending = False)
pd.set_option('display.min_rows', 50)
stock_data_df

,Ticker,Date,Price,Shares Outstanding,Market Cap,EV,EBITDA,EV/EBITDA,D/E,BV/Share,Total Cash,Total Liabilities,Total Revenue,Operating Cash Flow,BS Recent Date,CF Recent Date,IS Recent Date,Net Net per Share,NNPS less Price,NNPS less Price as % of Price
30,AEX.V,"04/19/2020, 23:59:16",0.000,70950000.000,41150000.000,39630000.000,-4920000.000,0.000,0.000,0.000,1515000.000,645000.000,0.000,-4158000.000,2019-12-31,2019-12-31,2019-12-31,0.012,0.012,inf
744,JDN.V,"04/20/2020, 04:01:35",0.015,91000000.000,1360000.000,1260000.000,0.000,0.000,0.000,0.000,20429000.000,173000.000,0.000,-286000.000,2018-12-31,2018-12-31,2018-12-31,0.223,0.208,13.840
174,BIRD-P.V,"04/20/2020, 00:48:18",0.010,14000000.000,0.000,0.000,0.000,0.000,0.000,0.000,772000.000,13000.000,0.000,0.000,2018-12-31,ttm,ttm,0.054,0.044,4.421
903,MMM.V,"04/20/2020, 04:55:29",0.060,50730000.000,3040000.000,-10010000.000,1760000.000,0.000,0.000,0.000,13075000.000,170000.000,2501000.000,1522000.000,2019-12-31,2019-12-31,2019-12-31,0.254,0.194,3.240
253,CCK-H.V,"04/20/2020, 01:15:04",0.015,31720000.000,0.000,0.000,0.000,0.000,0.000,0.000,1357000.000,18000.000,0.000,-239000.000,2019-09-30,2019-09-30,2019-09-30,0.042,0.027,1.814
249,CBV.V,"04/20/2020, 01:13:41",0.045,15900000.000,0.000,-1280000.000,0.000,0.000,0.000,0.000,1937000.000,159000.000,1385000.000,190000.000,2019-11-30,2019-11-30,2019-11-30,0.112,0.067,1.485
673,HTE-P.V,"04/20/2020, 03:37:48",0.020,11200000.000,0.000,0.000,0.000,0.000,0.000,0.000,558000.000,5000.000,0.000,-62000.000,2019-12-31,2019-12-31,2019-12-31,0.049,0.029,1.469
416,EECC-P.V,"04/20/2020, 02:10:58",0.020,6100000.000,0.000,0.000,0.000,0.000,0.000,0.000,324000.000,27000.000,0.000,-91000.000,2019-09-30,2019-09-30,2019-09-30,0.049,0.029,1.434
809,LBY.V,"04/20/2020, 04:23:40",0.040,77740000.000,3110000.000,-4190000.000,-1620000.000,0.000,0.000,0.000,7605000.000,184000.000,0.000,-2231000.000,2018-12-31,2018-12-31,2018-12-31,0.095,0.055,1.386
852,MAP.V,"04/20/2020, 04:38:17",0.030,59000000.000,2060000.000,-1360000.000,0.000,0.000,0.000,0.000,3752000.000,48000.000,0.000,-1059000.000,2019-04-30,2019-04-30,2019-04-30,0.063,0.033,1.093
